In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:

spark.sql("""create table if not exists f1_presentation.calculated_race_results
(race_year INT,
team_name STRING,
driver_id INT,
race_id INT,
position INT,
points INT,
calculated_points INT,
created_date TIMESTAMP,
updated_date TIMESTAMP
)
using delta
""")

In [0]:
spark.sql(f"""
              CREATE OR REPLACE TEMP VIEW race_result_updated
              AS
              SELECT races.race_year,
                     constructors.name AS team_name,
                     drivers.driver_id,
                     drivers.name AS driver_name,
                     races.race_id,
                     results.position,
                     results.points,
                     11 - results.position AS calculated_points
                FROM f1_processed.results 
                JOIN f1_processed.drivers ON (results.driver_id = drivers.driver_id)
                JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)
                JOIN f1_processed.races ON (results.race_id = races.race_id)
               WHERE results.position <= 10
                 AND results.file_date = '{v_file_date}'
""")

In [0]:
%sql
select count(*) from race_result_updated

In [0]:
spark.sql("""
Merge into f1_presentation.calculated_race_results as tgt
using race_result_updated as upd
on (tgt.race_id = upd.race_id and tgt.driver_id = upd.driver_id)
when matched then
update set tgt.calculated_points = upd.calculated_points,
tgt.points = upd.points,
tgt.position = upd.position,
tgt.updated_date = current_timestamp()
when not matched then
insert (race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points, created_date)
values (race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points, current_timestamp())
""")

In [0]:
%sql 
select count(*) from f1_presentation.calculated_race_results

In [0]:
%sql
create or replace temp view dominant_drivers
as 
select driver_name, count(*) as total_races, avg(calculated_points) as avg_points,
rank() over(order by avg(calculated_points) desc) driver_rank
from f1_presentation.calculated_race_results 
group by driver_name 
having total_races >= 50
order by avg_points desc

In [0]:
%sql
create or replace temp view dominant_teams
as 
select team_name, count(*) as total_races, avg(calculated_points) as avg_points,
rank() over(order by avg(calculated_points) desc) team_rank
from f1_presentation.calculated_race_results 
group by team_name
having total_races >= 100
order by avg_points desc

Visulaizations

In [0]:
%sql

select race_year, driver_name, count(*) as total_races, avg(calculated_points) as avg_points
from f1_presentation.calculated_race_results 
where driver_name in (select driver_name from dominant_drivers where driver_rank <= 10)
group by race_year, driver_name 
order by race_year, avg_points desc

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select race_year, team, count(*) as total_races, avg(calculated_points) as avg_points
from f1_presentation.calculated_race_results 
where team in (select team from dominant_teams where team_rank <= 10)
group by race_year, team
order by race_year, avg_points desc

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.